In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/barrettpoth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
stopwords = stopwords.words('english')[:]

In [4]:
pd.set_option("display.max_rows",270)

# Task A

### Engagement Score

In [5]:
post = pd.read_csv("insta_data_with_labels.csv").iloc[:,1:]
post.drop(columns='Unnamed: 0.1', inplace=True)

In [6]:
max_like = max(post["likes"])
max_like

1539774

In [7]:
max_like = max(post["likes"])
list_norm_like = []
for p in post.itertuples():
    norm_like = p.likes/max_like
    list_norm_like.append(norm_like)
post["norm_like"] = list_norm_like

max_comment = max(post["comments"])
list_norm_comment = []
for p in post.itertuples():
    norm_comment = p.comments/max_comment
    list_norm_comment.append(norm_comment)
    
post["norm_comment"]= list_norm_comment

In [8]:
engagement_score = []
for p in post.itertuples():
    e_score = .4*p.norm_like+.6*p.norm_comment
    engagement_score.append(e_score)

In [9]:
med_e_score = np.median(engagement_score)

In [10]:
eng_score = []
for i in range(len(engagement_score)):
    if engagement_score[i]>=med_e_score:
        e_score = 1
    else:
        e_score = 0
    eng_score.append(e_score)
eng_score
post["engagement_score"] = eng_score

In [11]:
pretty_post = post[["comments","likes","engagement_score"]]
pretty_post

,comments,likes,engagement_score
0,278,61459,0
1,362,83092,0
2,431,136026,0
3,1934,304484,1
4,907,413142,1
5,1495,431621,1
6,433,253932,0
7,2903,222257,0
8,590,100128,0
9,559,157608,0


# Task B

## Logistic regression

### Labels

In [12]:
post.dropna(axis=0, how='any', inplace=True)

In [13]:
label=post["labels"].values.tolist()
label=" ".join(label)
label = label.lower()        

In [14]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
labels = tokenizer.tokenize(label)

In [15]:
filter_labels = Counter(x for x in labels if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [16]:
filter_labels.most_common()[:25]

[('sky', 78),
 ('wildlife', 48),
 ('tree', 46),
 ('mammal', 45),
 ('water', 44),
 ('fauna', 39),
 ('mountain', 30),
 ('photography', 30),
 ('grass', 30),
 ('ecosystem', 29),
 ('wilderness', 28),
 ('animal', 28),
 ('organism', 27),
 ('rock', 24),
 ('terrestrial', 24),
 ('recreation', 22),
 ('nature', 22),
 ('girl', 22),
 ('plant', 21),
 ('elephant', 21),
 ('landscape', 20),
 ('bird', 19),
 ('darkness', 18),
 ('snout', 18),
 ('sea', 17)]

In [17]:
natural_objects=["wilderness","organism","autumn","spring","reflections","phenomenon","wood","nature","natrual",'penguins','cattle','species','insect','flower', 'whiskers','amphibian','fauna','elephant','animal','whale','bird','reptile','fish','grass','tree','plant','bird','cats','dog','horse','lions','bear','mammoths','elephants','wildlife','mammal',"tiger","snake"]
human = ["immigrants","population","dancer","male","female",'women','refugee',"snout",'chest','eye','hair','face','child','human',"family","children",'girl',"girls","people",'photojournalism',"community","communityies",'migrants',"man"]
geological  = ["desert","national park",'formation', 'geological','terrain','badlands',"landscape",'mountains',"mountain"'wilderness','forest','highland','grassland','hill','steppe','jungle',"vallay",'horizon','ecosystem','rock','terrestrial','landforms','landform','ecoregion',"geology","hillside"]
aerial =['cloud','sky','sunrise', 'star','atmosphere','aerial','light','darkness', 'sunset','night','astronomical' ,"sun","sunlight"]
activity = ["swimming","marriage","workshop",'festival','cycling','music','fun','climbing','religion','carnival','flame','ceremony','wedding', 'ritual','travel',"paragliding",'game', "recreation",'event','tradition','safari','vacation','sports','sport','competition event', 'boating',"boat"]
artificial_objects = ["pyramids","conservation","artificial",'painting','photography','sanctuary','aircraft',"food",'house','metropolis','outerwear', 'wall','design','room','arts','village','hall','fashion accessory','history', 'furniture','infrastructure', 'campfire','window','building','temple','city','street','vehicle','architecture','leisure','tourism','town','urban','computer','structure',"castle","cities"'town','landmark','town square', 'plaza', 'tourist attraction', 'road',"robots","transportation" ]
water=['island',"islands",'ocean','sea','iceberg', 'arctic ocean', 'shore','sea',"wave",'ice', 'shore','arctic','glacial ', 'polar ice cap', 'ice', 'melting', 'sea', 'ice cap','river','underwater',"coast","snow"]

In [18]:
def replacement(text):
    
    text = text.lower()
    
    for w in human:
        try:
            text = text.replace(w," human ")
        except:
            pass   
    for w in natural_objects:
        try:
            text = text.replace(w, ' natural ')
        except:
            pass
    for w in geological:
        try:
            text = text.replace(w, ' geological ')
        except:
            pass
    for w in aerial:
        try:
            text = text.replace(w, ' aerial ')
        except:
            pass
    for w in activity:
        try:
            text = text.replace(w, ' activity ')
        except:
            pass
    for w in water:
        try:
            text = text.replace(w, ' water ')
        except:
            pass

    for w in artificial_objects:
        try:
            text = text.replace(w, ' artificial ')
        except:
            pass
  

    return text

In [19]:
label_column = []
for p in post['labels']:
    new_text =replacement(p)
    label_column.append(new_text)

In [20]:
def cal_dfitf_score(text,min_df):
    vectorizer=CountVectorizer(stop_words="english", analyzer='word', min_df=min_df, max_features=None)
    transformer=TfidfTransformer()
    tfidf=transformer.fit_transform(vectorizer.fit_transform(text))
    word=vectorizer.get_feature_names()
    weight=tfidf.toarray()
    a=[]
    for i in range(len(weight)):
        a.append(weight[i])
    return a,word

In [21]:
a,word=cal_dfitf_score(label_column,0.1)

In [22]:
df=pd.DataFrame(a)
df.columns=word

In [23]:
df

,activity,aerial,artificial,geological,hu,human,natural,water
0,0.544354,0.000000,0.420734,0.000000,0.507174,0.493907,0.159662,0.000000
1,0.805181,0.000000,0.000000,0.593030,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.228391,0.923573,0.000000,0.000000,0.000000,0.000000,0.307977
3,0.000000,0.279709,0.282773,0.808381,0.000000,0.000000,0.214616,0.377177
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.943419,0.331602
5,0.000000,0.246649,0.000000,0.950447,0.000000,0.000000,0.189250,0.000000
6,0.385742,0.000000,0.894427,0.000000,0.000000,0.000000,0.226281,0.000000
7,0.265003,0.202603,0.819288,0.000000,0.000000,0.000000,0.466361,0.000000
8,0.000000,0.000000,0.000000,0.176546,0.000000,0.000000,0.984292,0.000000
9,0.000000,0.787009,0.000000,0.505448,0.000000,0.000000,0.000000,0.353750


In [24]:
df.drop(['hu'],axis=1,inplace=True)

In [25]:
y=post["engagement_score"]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
def logictic_pred(df,y):
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)
    lr = LogisticRegression() 
    lr.fit(X_train, y_train)
    y_pred=lr.predict(X_test)
    print("confusion_matrix:\n",confusion_matrix(y_test, y_pred))
    print("accuracy_score:\n",accuracy_score(y_test, y_pred))

In [27]:
logictic_pred(df,y)

confusion_matrix:
 [[14  9]
 [ 2 23]]
accuracy_score:
 0.7708333333333334


### Captions

In [28]:
text=post["captions"].values.tolist()
text=" ".join(text)
text = text.lower()
tokens = tokenizer.tokenize(text)

In [29]:
filter_words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [30]:
filter_words.most_common()[:300]

[('photo', 260),
 ('one', 89),
 ('see', 80),
 ('people', 80),
 ('follow', 77),
 ('years', 74),
 ('world', 73),
 ('natgeo', 65),
 ('many', 57),
 ('year', 56),
 ('like', 54),
 ('first', 53),
 ('wildlife', 50),
 ('time', 49),
 ('work', 47),
 ('life', 46),
 ('national', 45),
 ('new', 44),
 ('species', 41),
 ('part', 41),
 ('old', 40),
 ('around', 40),
 ('park', 40),
 ('back', 39),
 ('story', 38),
 ('china', 36),
 ('way', 35),
 ('still', 34),
 ('long', 34),
 ('wild', 34),
 ('women', 33),
 ('get', 32),
 ('conservation', 32),
 ('land', 32),
 ('made', 32),
 ('animals', 31),
 ('last', 29),
 ('day', 29),
 ('make', 28),
 ('also', 28),
 ('found', 28),
 ('kenya', 27),
 ('africa', 27),
 ('today', 27),
 ('images', 27),
 ('nature', 27),
 ('across', 27),
 ('find', 27),
 ('city', 25),
 ('war', 25),
 ('poaching', 25),
 ('live', 25),
 ('project', 25),
 ('assignment', 25),
 ('since', 25),
 ('children', 24),
 ('home', 24),
 ('elephants', 24),
 ('red', 24),
 ('water', 23),
 ('visit', 23),
 ('along', 22),
 ('

In [31]:
def clean_data(text):
    word=[]
    #delete punctuations stopwords
    words = re.compile(r'[^a-zA-Z]+').split(text)
    for w in words:
        if w not in stopwords and len(w)>2:
            word.append(w)
    return word

In [32]:
caption_column = []
for p in post['captions']:
    a=clean_data(p)
    a= ",".join(a)
    new_text =replacement(a) 
    caption_column.append(new_text)

In [33]:
b,w=cal_dfitf_score(caption_column,0.2)

In [34]:
df1=pd.DataFrame(b)
df1.columns=w

In [35]:
df1.drop(["hu","follow","years"],axis=1,inplace=True)

In [36]:
logictic_pred(df1,y)

confusion_matrix:
 [[17  6]
 [ 7 18]]
accuracy_score:
 0.7291666666666666


### labels+captions

In [37]:
post["all"]=post['captions']+post['labels']

In [38]:
all_column = []
for p in post['all']:
    a=clean_data(p)
    a= ",".join(a)
    new_text =replacement(a) 
    all_column.append(new_text)

In [39]:
c,z=cal_dfitf_score(all_column,0.2)

In [40]:
df2=pd.DataFrame(c)
df2.columns=z

In [41]:
df2.drop(["hu","follow","years","photo","world","like","natgeo"],axis=1,inplace=True)

In [42]:
logictic_pred(df2,y)

confusion_matrix:
 [[15  8]
 [ 1 24]]
accuracy_score:
 0.8125
